# Context

This notebooks shows how to calculate drawdowns, as an alternative to analyce the risk only whe negative deviations without considering the volatility given by positive deviations.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Read sample data of portafolio's returns
me_m = pd.read_csv('../input/python/Portfolios_Formed_on_ME_monthly_EW.csv',
                  header=0, index_col=0, parse_dates=True, na_values=99.99)

In [ ]:
# Convert index's datetype from integert to date (pd.to_datetime)
# Convert date to monthly periods (df.to_period)

me_m.index = pd.to_datetime(me_m.index, format="%Y%m").to_period("M")

In [ ]:
# Show only the low and high 10% that marketcap stocks
rets = me_m[['Lo 10','Hi 10']]

In [ ]:
# Rename the columns to SmallCap (Lo 10) and Large Caps (Hi 10)
rets.columns= ['SmallCap','LargeCap']

In [ ]:
# Normalice the returns
rets = rets/100

In [ ]:
rets.head()

In [ ]:
#Plot returns
rets.plot.line()

In [ ]:
# Show only the 1975's periods.
rets.loc['1975']

## Compute Drawdons

1. Compute a wealth index
2. Compute a previous peaks
3. Compute drawn as a percentage - Which is the wealth value as a percentaje of the previous peak

In [ ]:
# Create the wealth_index series and plot it, that is the cummulative product of the returns throught a given range of periods.
# To acchieve that, use numpy function cumprod().

wealth_index = 1000*(1+rets.loc[:,"LargeCap"]).cumprod()
wealth_index.plot.line()

In [ ]:
# Create the previous_peaks variable and plot it, that is the cummulative product of the max returns throught a given range of periods.
# To acchieve that, use numpy function cumprod().

previous_peaks = wealth_index.cummax()
previous_peaks.plot()

In [ ]:
# Calcualte the drawdown and plot it
drawdown = (wealth_index - previous_peaks) / previous_peaks
drawdown.plot()

In [ ]:
# IDentify the minimum drawdown at the series.
drawdown.min()

In [ ]:
# Identify the minimum drawdown for the 1975's year.
drawdown.loc["1975":].idxmin()

In [ ]:
# Identify the index of the minimum drawdown
drawdown.idxmin()

In [ ]:
#Create a function that let's calculate the drawdown for a given returns series.

def drawdown(returns_series: pd.Series):
    '''
    
    Takes a time series of asset returns
    computes and returns a DataFrame that contains:
    
    The wealth index
    the previous peaks
    percent drawdowns
    
    
    '''
    
    wealth_index = 1000*(1+returns_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    df = pd.DataFrame({
        "Wealth": wealth_index, 
        "Peaks": previous_peaks,
        "Drawdowns": drawdowns})
    return df
    

In [ ]:
# Create two series to compare the drawdowns.
rets_until_1950_lc = rets.loc[:'1950']['LargeCap']
rets_until_1950_sc = rets.loc[:'1950']['SmallCap']

In [ ]:
# Calculate the drawdowns for each series.
df_drawdown_before_1950_lc = drawdown(rets_until_1950_lc)
df_drawdown_before_1950_sc = drawdown(rets_until_1950_sc)

In [ ]:
# Plot the drawdowns for each series.
df_drawdown_before_1950_lc.plot(title="Large Caps Drawdowns")
df_drawdown_before_1950_sc.plot(title="Small Caps Drawdowns")

In [ ]:
# Identify the drawdown's minimum at the time when it happend for large and small caps"
print("Worse Drawdown large caps: \n", df_drawdown_before_1950_lc['Drawdowns'].min(), df_drawdown_before_1950_lc['Drawdowns'].idxmin())
print("Worse Drawdown small caps: \n", df_drawdown_before_1950_sc['Drawdowns'].min(), df_drawdown_before_1950_sc['Drawdowns'].idxmin())